# Decision Trees

In [1]:
import matplotlib.pyplot as plt
import numpy as np

## How is a Decision Tree fit?

* Which variables to include on the tree?
* How to choose the threshold?
* When to stop the tree?

**Key idea is that we want to choose the feature that has the lowest "impurity" to split our tree, thus our tree can reach the decision as fast as possible with smallest height possible.**

One way to measure impurity is using **Gini index** (another one is entropy but which measure very similar thing) with the following formula:

$$ I_{G} = 1 - \sum_{i=1}^{c} p_{i}^{2} $$

where $c$ is number of classes, and $p_{i}$ is the probability of each class.  For example, let's say our X is <code>[[2],[3],[10],[19]]</code> and y is <code>[0, 0, 1, 1]</code>.  That is, if a node has 4 samples, and 2 samples are of class cancer, and 2 samples are of no cancer, then the probability of each class is

$$p_{cancer}=(2/4)^2 = 0.25$$

$$p_{no_cancer}=(2/4)^2 = 0.25$$   

Thus the gini index of this node is

$$ I_{G} = 1 - (0.25 + 0.25) = 0.5 $$

Then we need to decide how to best split this node so we can get the lowest gini (highest purity) children.

For example, if we split this sample with $x < 3$: we will get left node X as <code>[[2]]</code> and y as <code>[0]</code> and the right node X as <code>[[3],[10],[19]]</code> and y as <code>[0, 1, 1]</code>.  The weighted gini of the children are 

$$ 1/4*I_{leftG} + 3/4 * I_{rightG} =  $$
$$ 1/4 * (1 - (1/1)^2) + 3/4 * (1 - (1/3)^2 - (2/3)^2) = 0.33 $$

Hmm...but we know we can split better, right?  Let's try $x < 4$: we will get left node X as <code>[[2],[3]]</code> and y as <code>[0, 0]</code> and the right node X as <code>[[10],[19]]</code> and y as <code>[1, 1]</code>.  If you do the math right, the gini is 0!

$$ 2/4 * (1 - (2/2)^2) + 2/4 * (1 - (2/2)^2 ) = 0 $$

Thus, in conclusion, we can say that spliting $x<4$ is a much better split than $x<3$.  In practice, to really find the best split, it is an exhaustive and greedy algorithm, in which we have to iterate and check every value on each feature as a candidate split, find the gini index.  

#### How do we find all threshold for continuous values?

We can first sort.  Then we are identify critical value using the midpoint between all consecutive values.  For example, given X is <code>[[2],[3],[10],[19]]</code>, the critical value to compare is 2.5, 6.5 and 14.5.

The code can be implemented in several ways.  Example are shown below:

In [2]:
X = np.array([[2], [3], [10], [19]])

In [3]:
X.shape

(4, 1)

In [4]:
'''
2.5, 6.5, 14.5

[2]    [3][10][19]
[0]    [0][1][1]

[2][3]   [10][19]
[0][0]   [1][1]

[2][3][10]  [19]
[0][0][1]   [1]
'''

'\n2.5, 6.5, 14.5\n\n[2]    [3][10][19]\n[0]    [0][1][1]\n\n[2][3]   [10][19]\n[0][0]   [1][1]\n\n[2][3][10]  [19]\n[0][0][1]   [1]\n'

In [5]:
def find_split(X, y, n_classes):
    
    m, n = X.shape  #samples, feature
    
    if m <= 1:
        return None, None
    
    feature_ix, threshold = None, None
    
    sample_per_class_parent = [np.sum(y == c) for c in range(n_classes)]  #[2, 2]
    
    #gini index of the parent
    best_gini = 1.0 - sum((m_i / m) ** 2 for m_i in sample_per_class_parent)
    
    for feature in range(n):
        
        sample_sorted = sorted(X[:, feature]) #[2, 3, 10, 19]
        sort_idx      = np.argsort(X[:, feature])
        y_sorted      = y[sort_idx] #[0, 0, 1, 1]
        
        sample_per_class_left  = [0] * n_classes #[0, 0]
        sample_per_class_right = sample_per_class_parent #[2, 2]
        
        for i in range(1, m):  #1, 2, 3  
            c = y_sorted[i - 1] #[0]
            
            sample_per_class_left[c]  += 1  #[1, 0]
            sample_per_class_right[c] -= 1  #[1, 2]
            
            gini_left = 1.0 - sum(
                (sample_per_class_left[x] / i) ** 2 for x in range(n_classes)
            )
            
            gini_right = 1.0 - sum(
                (sample_per_class_right[x] / (m - i)) ** 2 for x in range(n_classes)
            )
            
            weighted_gini = ((i / m) * gini_left) + ( (m - i) /m) * gini_right
            
            #in case the value are the same, we do not split
            #both have to end up on the same side of the split
            if sample_sorted[i] == sample_sorted[i - 1]:
                continue
            
            if weighted_gini < best_gini:
                best_gini = weighted_gini
                feature_ix = feature
                threshold = (sample_sorted[i] + sample_sorted[i -1]) / 2 
    
    return feature_ix, threshold

In [6]:
X = np.array([[2],[3],[10],[19]])
y = np.array([0, 0, 1, 1])

In [7]:
feature, threshold = find_split(X, y, len(set(y)))

In [8]:
feature

0

In [9]:
threshold

6.5

## Let's implement!

In [10]:
class Node:
    def __init__(self, gini, num_samples, num_samples_per_class, predicted_class):
        self.gini = gini
        self.num_samples = num_samples
        self.num_samples_per_class = num_samples_per_class
        self.predicted_class = predicted_class
        self.feature_index = 0
        self.threshold = 0
        self.left = None
        self.right = None

In [11]:
def fit(Xtrain, ytrain, n_classes, depth=0): 
    
    n_samples, n_features = Xtrain.shape
    num_samples_per_class = [np.sum(ytrain == i) for i in range(n_classes)]
    #predicted class using the majority of sample class
    predicted_class = np.argmax(num_samples_per_class) 
    
    #define the parent node
    node = Node(
        gini = 1 - sum((np.sum(y == c) / n_samples) ** 2 for c in range(n_classes)),
        predicted_class=predicted_class,
        num_samples = ytrain.size,
        num_samples_per_class = num_samples_per_class,
        )
    
    #perform recursion
    feature, threshold = find_split(Xtrain, ytrain, n_classes)
    
    if feature is not None:
        indices_left = X[:, feature] < threshold
        X_left, y_left   = X[indices_left], y[indices_left]
        X_right, y_right = X[~indices_left], y[~indices_left]
        
        #take note for later decision
        node.feature_index = feature
        node.threshold = threshold
        node.left  = fit(X_left, y_left, n_classes, depth + 1)
        node.right = fit(X_right, y_right, n_classes, depth + 1)
    
    return node

In [12]:
#to predict, it is as simple as moving 
#through the tree 
def predict(sample, tree):
    while tree.left:
        if sample[tree.feature_index] < tree.threshold:
            tree = tree.left
        else:
            tree = tree.right
    return tree.predicted_class


In [13]:
#fit starting with tree depth = 0
Xtrain = np.array([[2, 5],[3, 5],[10, 5],[19, 5]])
ytrain = np.array([0, 0, 1, 1])
Xtest = np.array(([[4, 6],[6, 9],[9, 2],[12, 8]]))
ytest = np.array([0, 0, 1, 1])

In [14]:
tree = fit(Xtrain, ytrain, len(set(ytrain)))

In [15]:
tree

In [16]:
pred = [predict(x, tree) for x in Xtest]

In [17]:
pred

[0, 0, 1, 1]

In [18]:
print("Tree feature ind: ", tree.feature_index)
print("Tree threshold: ", tree.threshold)
print("Pred: ", np.array(pred))
print("ytest: ", ytest)

Tree feature ind:  0
Tree threshold:  6.5
Pred:  [0 0 1 1]
ytest:  [0 0 1 1]


In [ ]:
#decision is prone to overfitting